In [ ]:
import os
import pandas as pd
import random
import skimage.data
import skimage.transform
#import opencv_python
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image

In [ ]:
#pre_processing
for a in range(len(train_shape)):
        train_shape[a]=int(train_shape[a])-1

for a in range(len(test_shape)):
        test_shape[a]=int(test_shape[a])-1

#reshape the image
image_train_24=[skimage.transform.resize(image, (24, 24)) for image in np.array(train_objects)]
image_test_24=[skimage.transform.resize(image, (24, 24)) for image in np.array(test_objects)]



In [ ]:
learning_rate = 1e-4
training_step = 2500
display_step = 50

images_train=image_train_24
labels_train=train_shape
images_test=image_test_24
labels_test=np.array(test_shape)

#Defining placeholders
input_x = tf.placeholder(dtype=tf.float32,shape=[None,24,24,3])   #图像大小24x24x3
input_y = tf.placeholder(dtype=tf.float32,shape=[None,10])        #,类别 

x_image = tf.reshape(input_x,[-1,24,24,3])

#1.Convolution layer -> pooling layer
h_conv1 = tf.contrib.layers.conv2d(inputs=x_image,num_outputs=32,kernel_size= 5,stride=1,padding='SAME', activation_fn=tf.nn.relu)    #输出为[-1,32,32,32]
h_pool1 = tf.contrib.layers.max_pool2d(inputs=h_conv1,kernel_size=2,stride=2,padding='SAME')         #输出为[-1,16,16,32]


#2.Convolution layer -> pooling layer
h_conv2 =tf.contrib.layers.conv2d(inputs=h_pool1,num_outputs=64,kernel_size=[5,5],stride=[1,1],padding='SAME', activation_fn=tf.nn.relu)    #输出为[-1,16,16,64]
h_pool2 =  tf.contrib.layers.max_pool2d(inputs=h_conv2,kernel_size=[2,2],stride=[2,2],padding='SAME')   #输出为[-1,8,8,64]


#3.Fully connected layer
nt_hpool2 = tf.contrib.layers.avg_pool2d(inputs=h_pool2,kernel_size=6,stride=6,padding='SAME')          #输出为[-1,1,1,64]
nt_hpool2_flat = tf.reshape(nt_hpool2,[-1,64])            
y_conv = tf.contrib.layers.fully_connected(inputs=nt_hpool2_flat,num_outputs=10,activation_fn=tf.nn.softmax)

#prediction
predicted_labels = tf.argmax(y_conv, 1)

#Softmax cross entropy cost function
cost = tf.reduce_mean(-tf.reduce_sum(input_y * tf.log(y_conv),axis=1))

#solver
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#Return an accuracy data
correct_prediction = tf.equal(tf.arg_max(y_conv,1),tf.arg_max(input_y,1))
#Accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))

#train
sess = tf.Session();
sess.run(tf.global_variables_initializer())

# Start all the queue threads in the calculation graph.。
tf.train.start_queue_runners(sess=sess)

for step in range(training_step):
    #one hot
    labels_train=labels_train.astype(int)
    labels_test=labels_test.astype(int)
    labels_b = np.eye(10,dtype=float)[labels_train]
    labels_a=np.eye(10,dtype=float)[labels_test]
    
    #Start training
    train.run(feed_dict={input_x:images_train,input_y:labels_b},session=sess)
    
    #print accuracy
    if step % display_step == 0:
        train_accuracy = accuracy.eval(feed_dict={input_x:images_train,input_y:labels_b},session=sess)
        test_accuracy = accuracy.eval(feed_dict={input_x:images_test,input_y:labels_a},session=sess)
        print('Step {0} training accuracy {1}'.format(step,train_accuracy))
        print('Step {0} testing accuracy {1}'.format(step,test_accuracy))

In [ ]:
#Calculate the accuracy of the test set
predict=sess.run([predicted_labels],feed_dict={input_x:images_test})[0]
match_count = sum([int(y == y_) for y, y_ in zip(labels_test, predict)])
accuracy = match_count / len(labels_test)
print("Accuracy: {:.3f}".format(accuracy))

In [ ]:
#Randomly select 20 images in the test set to test their accuracy
sample_indexes = random.sample(range(len(image_test)), 20)
sample_images = [images_test[i] for i in sample_indexes]
sample_labels = [labels_test[i] for i in sample_indexes]

# Run the "predicted_labels" op.
predicted = sess.run([predicted_labels], feed_dict={input_x: sample_images})[0]

fig = plt.figure(figsize=(10, 10))
for i in range(len(sample_images)):
    truth = sample_labels[i]
    prediction = predicted[i]
    plt.subplot(10, 2,1+i)
    plt.axis('off')
    color='green' if truth == prediction else 'red'
    plt.text(40, 10, "Truth:        {0}\nPrediction: {1}".format(truth, prediction), fontsize=12, color=color)
    plt.imshow(sample_images[i])